In [1]:
import os
import json
import random
import numpy as np
import tensorflow as tf

In [ ]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the senses to be used as inputs for training
x_train_sense = []
# List that holds all the locals to be used as inputs for training
x_train_locals = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the sense to be used as inputs for testing
x_test_sense = []
# List that holds all the locals to be used as inputs for testing
x_test_locals = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [ ]:
# Name of directory for grids
directory_name = './data/big_3/'

direct_count = [0 for i in range(4)]

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_sense.append(i['sense'])
        x_train_locals.append(i['local'])
        y_train.append(i['direction'])
        direct_count[i['direction']] += 1
    
    # Close file socket
    f.close()

In [ ]:
# Name of directory for grids
directory_name = './data/big_3test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_sense.append(i['sense'])
        x_test_locals.append(i['local'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [ ]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 50, 50, 1) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 50, 50, 1) )
train_in_sense = np.reshape( x_train_sense, (-1, 50, 50) ) / 8
train_in_locals = np.reshape( x_train_locals, (-1, 5, 5, 1) )
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 50, 50, 1) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 50, 50, 1) )
test_in_sense = np.reshape( x_test_sense, (-1, 50, 50) ) / 8
test_in_locals = np.reshape( x_test_locals, (-1, 5, 5, 1) )
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [ ]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (50,50,1) )
cnn_grid = tf.keras.layers.Conv2D( filters = 4, kernel_size = (5,5), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( grid_input )
flatten_grid = tf.keras.layers.Flatten()( cnn_grid )

2021-12-15 00:20:16.584954: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (50,50,1) )
cnn_position = tf.keras.layers.Conv2D( filters = 2, kernel_size = (5,5), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( position_input )
flatten_position = tf.keras.layers.Flatten()( cnn_position )

In [ ]:
# Flatten the position input
sense_input = tf.keras.layers.Input( shape = (50,50,1) )
cnn_sense = tf.keras.layers.Conv2D( filters = 8, kernel_size = (5,5), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( sense_input )
flatten_sense = tf.keras.layers.Flatten()( cnn_sense )

In [ ]:
# flatten the local inputs
local_input = tf.keras.layers.Input( shape = (5,5,1) )
cnn_local = tf.keras.layers.Conv2D( filters = 3, kernel_size = (3,3), strides = (1,1),
                               padding = "valid", activation = tf.nn.relu )( local_input )
flatten_local = tf.keras.layers.Flatten()( cnn_local )

In [ ]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position, flatten_sense, flatten_local])
print(final_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 29651), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")


In [ ]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( dense_1 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )
probabilities = tf.keras.layers.Softmax()( logits )

In [ ]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input, sense_input, local_input], outputs = probabilities )
opt = tf.keras.optimizers.SGD()
model.compile( optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    prob_predict = model.predict( data )
    #print(prob_predict)
    predictions = np.argmax( prob_predict, axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
        # if labels[i] == 0:
        #     print(prob_predict[i])
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [ ]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals], y_test )

0	121	0	130
0	270	0	376
0	420	6	1100
0	533	9	1540


In [ ]:
# Train the model
history = model.fit( [train_in_grid, train_in_position, train_in_sense, train_in_locals], train_out, validation_data=([test_in_grid, test_in_position, test_in_sense, test_in_locals], test_out), epochs = 5 )

Epoch 1/5
1456/1456 [==============================] - 93s 64ms/step - loss: 0.2131 - accuracy: 0.9254 - val_loss: 0.1244 - val_accuracy: 0.9614
Epoch 2/5
1456/1456 [==============================] - 105s 72ms/step - loss: 0.0962 - accuracy: 0.9670 - val_loss: 0.0919 - val_accuracy: 0.9687
Epoch 3/5
1456/1456 [==============================] - 120s 83ms/step - loss: 0.0784 - accuracy: 0.9729 - val_loss: 0.0924 - val_accuracy: 0.9698
Epoch 4/5
1456/1456 [==============================] - 123s 84ms/step - loss: 0.0680 - accuracy: 0.9767 - val_loss: 0.0744 - val_accuracy: 0.9771
Epoch 5/5
1456/1456 [==============================] - 124s 85ms/step - loss: 0.0615 - accuracy: 0.9783 - val_loss: 0.1049 - val_accuracy: 0.9689


In [ ]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_sense, test_in_locals,], y_test )

238	4	5	4
51	557	7	31
13	3	1499	11
10	0	1	2071


In [ ]:
model.save('./models/agent3_CNN')

2021-12-15 00:29:52.326289: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/agent3_CNN/assets
